In [1]:
# POS tagging and NER

import spacy
nlp = spacy.load('en_core_web_sm')

In [2]:
doc = nlp(u'The quick brown fox jumped over the lazy fox.')

In [13]:
for token in doc:
    print(f"{token.text:{10}} {token.pos_:{5}} {token.tag_:{5}} {spacy.explain(token.tag_)}")

The        DET   DT    determiner
quick      ADJ   JJ    adjective
brown      ADJ   JJ    adjective
fox        NOUN  NN    noun, singular or mass
jumped     VERB  VBD   verb, past tense
over       ADP   IN    conjunction, subordinating or preposition
the        DET   DT    determiner
lazy       ADJ   JJ    adjective
fox        NOUN  NN    noun, singular or mass
.          PUNCT .     punctuation mark, sentence closer


In [30]:
# POS counts 
pos_counts = doc.count_by(spacy.attrs.POS)

In [51]:
pos_counts  
# returns a dict of POStag index in the vocab and the count of it

{96: 1, 83: 3, 99: 1, 84: 1, 89: 2, 91: 2}

In [52]:
print(doc[2].text, doc[2].pos, doc[2].pos_)

brown 83 ADJ


In [53]:
doc.vocab[83].text  
# 83 is an ADJ and there are 3 of them in the sentence


'ADJ'

In [56]:
# frequency of pos tags

for k,v in sorted(pos_counts.items()):
    print(f'{k}. {doc.vocab[k].text:{6}} {v}')

83. ADJ    3
84. ADP    1
89. DET    2
91. NOUN   2
96. PUNCT  1
99. VERB   1


In [57]:
TAG_counts = doc.count_by(spacy.attrs.TAG)

for k,v in sorted(TAG_counts.items()): 
    print(f'{k}. {doc.vocab[k].text:{6}} {v}')



1292078113972184607. IN     1
10554686591937588953. JJ     3
12646065887601541794. .      1
15267657372422890137. DT     2
15308085513773655218. NN     2
17109001835818727656. VBD    1


In [58]:
DEP_counts = doc.count_by(spacy.attrs.DEP)

In [62]:
for k,v in sorted(DEP_counts.items()):
    print(f'{k}. {doc.vocab[k].text:{6}} {v}')

399. amod   3
412. det    2
426. nsubj  1
436. pobj   1
440. prep   1
442. punct  1
8206900633647566924. ROOT   1


In [ ]:
# Visualizing POS 

